<a href="https://colab.research.google.com/github/yunaficus/DRL/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!git clone https://github.com/OptimalPursuit/navigating_uncertainty_in_mpp.git

Cloning into 'navigating_uncertainty_in_mpp'...
remote: Enumerating objects: 12419, done.
remote: Counting objects: 100% (619/619), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 12419 (delta 435), reused 603 (delta 434), pack-reused 11800 (from 1)
Receiving objects: 100% (12419/12419), 578.14 MiB | 33.53 MiB/s, done.
Resolving deltas: 100% (8402/8402), done.
Updating files: 100% (1357/1357), done.


In [25]:
cd navigating_uncertainty_in_mpp

[Errno 2] No such file or directory: 'navigating_uncertainty_in_mpp'
/content/navigating_uncertainty_in_mpp


In [11]:
!pip install -r requirements.txt

  Cloning https://github.com/ai4co/rl4co.git (to revision 907d07567bc06813ddbd222a13c49682ff1d1b63) to /tmp/pip-install-2puj1b26/rl4co_0049ecce001d416eaa35918c4e5dec75
  Running command git clone --filter=blob:none --quiet https://github.com/ai4co/rl4co.git /tmp/pip-install-2puj1b26/rl4co_0049ecce001d416eaa35918c4e5dec75
  Running command git rev-parse -q --verify 'sha^907d07567bc06813ddbd222a13c49682ff1d1b63'
  Running command git fetch -q https://github.com/ai4co/rl4co.git 907d07567bc06813ddbd222a13c49682ff1d1b63
  Running command git checkout -q 907d07567bc06813ddbd222a13c49682ff1d1b63
  Resolved https://github.com/ai4co/rl4co.git to commit 907d07567bc06813ddbd222a13c49682ff1d1b63
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/DLR-RM/stable-baselines3 (to revision 6ad6fa55b6e38c8456dd333f71fe45373f66fe90) to /tmp/pip-install-2puj1b26/stable-baselines3_63a54ab04b174dac

In [26]:
!pip install cplex==22.1.1.1


In [27]:
!python main.py  # Modify based on actual usage

/usr/local/lib/python3.11/dist-packages/torchrl/data/replay_buffers/samplers.py:34: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. This is likely due to a discrepancy between your package version and the PyTorch version. Make sure both are compatible. Usually, torchrl majors follow the pytorch majors within a few days around the release. For instance, TorchRL 0.5 requires PyTorch 2.4.0, and TorchRL 0.6 requires PyTorch 2.5.0.
  warnings.warn(EXTENSION_WARNING)
Running with folder: sac-fr, algorithm type: sac,generalization: False
2025-06-03 16:09:22,175 [torchrl][INFO] check_env_specs succeeded!
Warm-up: 100% 3/3 [00:04<00:00,  1.50s/it]
Episodes:   0% 0/30 [00:00<?, ?it/s]Max Revenue: 1700.6063232421875
Episodes:   3% 1/30 [00:01<00:35,  1.21s/it]Max Revenue: 1865.72607421875
Episodes:   7% 2/30 [00:02<00:33,  1.21s/it]Max Revenue: 1798.8203125
Episodes:  10% 3/30 [00:03<00:32,  1.21s/it]Max Revenu


一、研究背景與動機
---
全球超過45%的貨物是透過貨櫃運輸完成，總值高達8.1兆美元，然而貨櫃船的艙位配置（stowage planning）受到載重限制、穩定性、安全與需求變動等多重限制，因此規劃極為複雜。傳統方法難以應對這些不確定因素且運算成本高昂。

為解決此問題，本文提出一種結合 深度強化學習（Deep Reinforcement Learning, DRL）與可行性投影機制（feasibility projection） 的方法，專注於貨櫃主配置規劃問題（Master Stowage Planning Problem, MPP），目標為在面對不確定需求下，提升貨櫃艙位使用效率與營收。

---
二、研究貢獻
---
🔧 建立真實場域的 MDP 模型：將貨櫃主配置規劃建模為一個 Markov 決策過程，模擬不同港口的動態變化與艙位限制。

🧩 引入可微分的可行性投影層（Projection Layers）：包含權重縮放（WS）、策略剪裁（PC）、違規修正（VP）三種方式，有效強化模型對限制條件的學習與滿足能力。

⚙️ 提出 Encoder-Decoder 架構的 Actor-Critic 強化學習方法：搭配動態與情境嵌入（embedding）方式，讓模型能前瞻性地判斷未來狀況。

📈 實驗證明效能優於傳統方法：包括混合整數規劃（SMIP）與原生強化學習法（PPO、SAC），在可行性與營收表現上均有顯著提升。

---
三、方法架構概述
---
為了解決貨櫃主配置規劃問題（MPP）中動作空間複雜、限制條件多且需求不確定的挑戰，本文提出一個結合「深度強化學習（Deep Reinforcement Learning, DRL）」與「可行性投影機制（Feasibility Projection）」的架構，流程如下

Cargo Embedding → Encoding →

Context / Dynamic Embedding → Actor & Critic

動作輸出 → 可行性投影修正

評估價值 → 回傳訓練信號更新模型

整體方法主要包含三個核心組件：

1. Encoder-Decoder 架構（以注意力機制為核心）
🔹 Cargo Embedding
將每一筆貨櫃資料（包括來源港、目的港、貨櫃大小、重量、合約類型等）轉換為數值特徵向量，再加上位置編碼（Positional Encoding）來保留順序資訊，方便後續模型捕捉時間與空間上下文。

🔹 Encoder
透過多頭注意力機制（Multi-Head Attention, MHA），將貨櫃特徵進行全局建模，得到代表整體航程需求與結構的潛在表示（Latent Representation）。

🔹 Context Embedding 與 Dynamic Embedding
Context Embedding 根據當下船艙使用情況 ut 與編碼結果 z，生成上下文向量，幫助政策關注當前的船況與艙位限制。

Dynamic Embedding 則基於未來需求變化（qt）與時間步長進行序列建模，捕捉長期需求變化趨勢。

🔹 Decoder (Actor-Critic)
Actor 根據當前狀態與嵌入資訊，使用注意力指標機制（Pointer Network）輸出行動分布：πθ(x|s)，採樣動作 xt。

Critic 為單層或雙層 FFN，用來估算當前狀態的期望價值 V(st)。

2. 可行性投影機制（Feasibility Projection Layers）
由於 MPP 問題具有明確的線性不等式限制（如：容量限制、穩定性限制、需求限制等），必須確保強化學習輸出的行動能夠落在可行域中。因此作者設計三種可微分的投影方法：

🔸 Weighted Scaling (WS)
當行動向量總和超過上限（如需求量或 TEU 容量），則按比例縮放，使其剛好符合限制，並保留原始比例。

🔸 Policy Clipping (PC)
將每個動作元素限制在上下界區間內，類似於 box constraints 處理（element-wise bounding）。

🔸 Violation Projection (VP)
若行動違反某些約束條件，則透過梯度下降方式將其投影回可行域的近似點，並利用 Jacobian 調整保留 log 機率密度。

這些機制可以視為類似 Soft Constraint 的可行性修復策略，使學習過程中的動作不至於因違規而導致不穩定訓練。

3. 兩階段馬可夫決策過程設計（Formal MDP 與 Decomposed MDP）
為了提高訓練效率，該研究將原始 MDP 分為兩種形式處理：

📌 Formal MDP
每一港口一次性做出全部艙位分配決策（高維度）

動作空間為 |B| × |D| × |K| × |TR|，容易導致學習不穩定

📌 Decomposed MDP
將一個港口的決策細分為多個小步驟，逐步處理每個 (pol, pod, cargo type)

將動作空間降為 |B| × |D|，延長時間步長並提升學習穩定性

成本函數僅在港口階段進行評估（sparse reward）

✨ 損失函數與訓練方式
強化學習採用 Actor-Critic 框架，具體損失函數如下：

Actor Loss（策略最大化 + 可行性懲罰）
Critic Loss（狀態價值函數的 MSE）
Feasibility Loss

---
四、實驗設計與結果
---
測試項目：
🚢 載重 1000 TEU，4 個港口的航線

模擬不確定需求（高/低變異）

比較 DRL（SAC/PPO）與 SMIP 方法

評估指標：利潤（Ob.）、計算時間、可行性（F.I.）、違規程度（d(PH)）

| 方法                | 可行性 (%) | 平均利潤 (\$)   | 計算時間 (秒)    |
| ----------------- | ------- | ----------- | ----------- |
| SAC + 投影層 (WS/PC) | 100     | **1494.22** | **13.12**   |
| PPO + 投影層 (WS/PC) | 100     | 1471.40     | 13.75       |
| 傳統 SMIP-NA        | 100     | 1053.02     | **8434.58** |


SAC 與 PPO 在加入投影後皆可保證100%可行性，且獲得最高利潤。

相比之下，SMIP-NA 雖能解出解，但計算時間太長，不適用於實務。

---
五、管理意涵與實務建議
---
✅ 可行性機制效果佳：權重縮放與投影層能有效取代繁複的 Lagrangian 式懲罰。

⚡ 可大幅降低計算成本：投影後 DRL 推理僅需數十秒，適合實務快速調度需求。

🔄 對不確定性具適應能力：DRL 模型在需求變異越高時反而表現更佳，具預測與回應能力。

📊 可推廣至實務部署：未來實作應重視收集代表性需求數據，以提升模型泛化能力。

---
六、結論與未來工作
---
本研究展示了深度強化學習結合可行性投影在貨櫃船配置規劃上的潛力，能即時提供動態決策支援。未來方向包含：

擴展模型至更大船隻與更長航線

結合真實操作資料進行驗證

與其他港口操作（如吊車排程）整合進行聯合最佳化

